Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
monitors = ["a.1", "a.2", "a.3", "a.4", "a.5", "a.6", "a.7", "bp", "bpC"],
model=m_10_04, output_format=:mcmcchains);

Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
      mean   sd  5.5% 94.5% n_eff Rhat
a[1] -0.74 0.27 -1.17 -0.31  3838    1
a[2] 11.02 5.53  4.46 21.27  1759    1
a[3] -1.05 0.28 -1.50 -0.61  3784    1
a[4] -1.05 0.27 -1.48 -0.62  3761    1
a[5] -0.74 0.27 -1.18 -0.32  4347    1
a[6]  0.21 0.27 -0.23  0.66  3932    1
a[7]  1.81 0.39  1.19  2.46  4791    1
bp    0.84 0.26  0.42  1.26  2586    1
";

Describe the draws

In [8]:
describe(chn)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = Chain1, Chain2, Chain3, Chain4
Samples per chain = 1000
parameters        = a.1, a.2, a.3, a.4, a.5, a.6, a.7, bp, bpC

Empirical Posterior Estimates:
parameters
      Mean    SD   Naive SE  MCSE   ESS
a.1 -0.7353 0.2706   0.0043 0.0046 1000
a.2 10.8091 5.2227   0.0826 0.1126 1000
a.3 -1.0468 0.2789   0.0044 0.0039 1000
a.4 -1.0447 0.2759   0.0044 0.0052 1000
a.5 -0.7349 0.2727   0.0043 0.0033 1000
a.6  0.2241 0.2717   0.0043 0.0046 1000
a.7  1.8011 0.3920   0.0062 0.0052 1000
 bp  0.8341 0.2614   0.0041 0.0045 1000
bpC -0.1353 0.3012   0.0048 0.0054 1000

Quantiles:
parameters
      2.5%   25.0%   50.0%   75.0%   97.5% 
a.1 -1.6318 -0.9158 -0.7322 -0.5510  0.2052
a.2  2.3568  6.8749  9.6617 13.6518 35.1520
a.3 -1.9931 -1.2381 -1.0444 -0.8563  0.0858
a.4 -2.1465 -1.2267 -1.0364 -0.8615 -0.1541
a.5 -1.7652 -0.9164 -0.7340 -0.5503  0.2456
a.6 -0.7299  0.0486  0.2154  0.4019  1.3045
a.7  0.5065  1

End of `10/m10.02s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*